# Run full Granual Predictions

What needs to happen:
- ~~function to temporally and spatially interpolate MERRA2~~
- ~~function to read MERRA2 with interpolation~~
  - PS = surface_pressure
  - T10M = 10-meter_air_temperature (Andy will investigate)
  - TO3 = total_column_ozone
  - TQV = total_precipitable_water_vapor
- ~~function to read VNP02MOD~~
  - in group "observation_data"
    - M14 (for center wavelengths at 8500 nm)
    - M15 (for center wavelengths at 10800 nm)
    - M16 (for center wavelengths at 12000 nm)
- ~~function to read VNP03MOD~~
  - in group "geolocation_data"
    - sensor_azimuth
    - sensor_zenith
    - solar_azimuth
    - solar_zenith
- combine above for model inputs
- check if there are any transformations that need to be made on the inputs before sening them though the model
- load the saved model (see tensorflow.keras.models.load_model as in evaluate.ipynb)
- call the saved model's "predict" method on the combined inputs

## Setup

In [ ]:
from pathlib import Path
from importlib import reload

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from hot_dust import preprocess, DATADIR

In [ ]:
preprocess = reload(preprocess)

## Load the VIIRS Granules

In [ ]:
# granules
vnp02 = "VNP02MOD.A2020168.1448.002.2021127084950.nc"
vnp03 = "VNP03MOD.A2020168.1448.002.2021125194020.nc"

In [ ]:
# open WITHOUT mask_and_scale to access observations as integers (ignore scale and offset)
open_vnp02_int = xr.open_dataset(
    DATADIR / "granules" / vnp02,
    group="observation_data",
    mask_and_scale=False,
)

# open WITH mask_and_scale to access LUT values as floats (with NaNs)
open_vnp02 = xr.open_dataset(
    DATADIR / "granules" / vnp02,
    group="observation_data",
)

# create new xr.Dataset to contain viirs_bts
vnp02_variables = xr.Dataset()
for item in ["M14", "M15", "M16"]:
    vnp02_variables[item + "_bt"] = open_vnp02[item + "_brightness_temperature_lut"][open_vnp02_int[item]]
    long_name = vnp02_variables[f"{item}_bt"].attrs["long_name"]
    vnp02_variables[item + "_bt"].attrs["long_name"] = long_name.replace(" lookup table", "")

In [ ]:
# open (just open, no tricks here)
open_vnp03 = xr.open_dataset(
    DATADIR / "granules" / vnp03,
    group="geolocation_data",
)

# select the relavant variables
vnp03_variables = open_vnp03[["sensor_zenith", "solar_zenith"]]

# compute relative azimuth
array = np.abs(open_vnp03["solar_azimuth"] - open_vnp03["sensor_azimuth"])
vnp03_variables["relative_azimuth"] = array.where(array <= 180, 360 - array)

In [ ]:
# merge the model imputs
vnp02_vnp03 = xr.merge([vnp02_variables, vnp03_variables])
vnp02_vnp03

## Extract MERRA-2 Input Variables

The "raw" cell below has code to read the MERRA2 files directly from AWS, so it will not run locally. The output has been commited to DVC, however, so it does not need to be re-run unless the preprocessing code changes.

In [ ]:
# Merge all the merra files
merra_variables = xr.open_dataset(DATADIR / 'merra' / vnp03) 

# Drop the coordinates 
merra_variables = merra_variables.drop(['time', 'lon', 'lat'])

# Divide the MERRA2 pressure by 100 to get it in the right units
merra_variables['PS'] = merra_variables['PS']/100 
merra_variables

In [ ]:
# Merge viirs and merra variables
variables_merged = xr.merge([vnp02_vnp03, merra_variables]) 

## 2D Plot of Input Variables

In [ ]:
# M14 Map (red)
plt.imshow(variables_merged["M14_bt"], cmap="Reds", interpolation="none")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()  

In [ ]:
# M15 Map (green)
plt.imshow(variables_merged["M15_bt"], cmap="Greens", interpolation="none")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# M16 Map (blue)
plt.imshow(variables_merged["M16_bt"], cmap="Blues", interpolation="none")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Solar zenith angle
plt.imshow(variables_merged["sensor_zenith"], cmap="Grays")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Solar zenith angle
plt.imshow(variables_merged["PS"], cmap="Grays")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Solar zenith angle
plt.imshow(variables_merged["TQV"], cmap="Grays")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Solar zenith angle
plt.imshow(variables_merged["WS"], cmap="Grays")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

In [ ]:
# Solar zenith angle
plt.imshow(variables_merged["TS"], cmap="Grays")
# Colorbar and lables
cb = plt.colorbar(shrink=0.5)
plt.show()

## Transform to Model Input

In [ ]:
# the features list must give the same data as the features list in hot_dust.preprocess.prepare_training_data
features = [
#    "solar_zenith",
    "sensor_zenith",
#    "relative_azimuth",
    "PS",
    "TQV",
    "TO3",
    "WS",
    "TS",
    "M14_bt",
    "M15_bt",
    "M16_bt",
]

# Concatenate the feature variables, in the order above, into a 3D array
x = variables_merged[features].to_array("feature")
# Stack number of lines and pixels to make it a 2D array with 'sample' and 'feature'
x = x.stack({"sample": ('number_of_lines', 'number_of_pixels')})
x = x.dropna(dim="sample")
x

## Generate Predictions

## 2D Plot of Predicted Dust Optical Thickness

## WIP / Scratch

In [ ]:
xr.open_dataset("data/rt_nn_irdust_training_data.nc")

In [ ]:
xr.open_dataset("data/granules/GMAO_MERRA2.20200616T140000.MET.nc")